In [131]:
#generate for bar chart
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import word2vec
import json
import numpy as np
from konlpy.tag import Kkma
from konlpy.utils import pprint
import math
import csv
import codecs
l_quality= open("lecture_quality.txt", "r", encoding="utf-8").readlines()
load= open("load.txt", "r", encoding="utf-8").readlines()
grade= open("grade.txt", "r", encoding="utf-8").readlines()
int_friendly= open("international_friendly.txt", "r", encoding="utf-8").readlines()
CATEGORY_COUNT = 3

In [132]:
pos_vectors = KeyedVectors.load_word2vec_format('../pos_naver_news.vec', binary=False)

In [133]:
scores = []
subjects = [l_quality, load, grade]
def add_to_dict(subject):
    subject_scores={}
    for i in range(len(subject)):
        p = subject[i].split("\t")
        subject_scores[p[0]] = float(p[1][:-1])
    scores.append(subject_scores)

for subject in subjects:
    add_to_dict(subject)

In [134]:
subject_mp = {"강의":0,"널널":1,"성적":2}
def calculate_score(s, subject_str):
    result = 0
    for k,v in scores[subject_mp[subject_str]].items():
        if k in s:
            result += v
#     result=1.0/(1+math.exp(-result))
    result=math.tanh(result)
    return result

In [135]:
REVIEWS = "cs_reviews"

f = open("../reviews_in_one_txt/" + REVIEWS + ".txt","r",encoding="utf-8").readlines()

scores_list = []
subjects_str = ["강의", "널널", "성적"]
for subject_str in subjects_str:
    scores_list_subject=[]
    for i in range(len(f[:])):
        line = f[i]
        course = json.loads(line)
        scores_list_subject.append(calculate_score(course["review_text"], subject_str))
    scores_list.append(scores_list_subject)


In [136]:
f_count_word = {}
count_word = {}
mp = {"A":5,"B":4,"C":3,"D":2,"F":1}

def calc_weight(s):
    return 3 - (mp[s])

for i in range(len(f)):
    line = f[i]
    course = json.loads(line)
    if course["강의"] == "?":
        continue
    words = course["review_text"].split()
    for w in words:
        if w not in count_word:
            count_word[w] = 0
        count_word[w] += calc_weight(course["강의"])


In [137]:
sorted_by_value = sorted(count_word.items(), key=lambda kv: kv[1])[::-1]

In [138]:
# sorted_by_value

In [139]:
def calc_score_avg(column_name,comment_id):
    i = comment_id
    course = json.loads(f[i])
    #print(mp[course["강의"]],(scores_list[i]))
    return scores_list[subject_mp[column_name]][i]
    

for i in range(len(f)):
    course = json.loads(f[i])
    print(course["course_code"])
    
    for x in subjects_str:
        print(x,calc_score_avg(x,i))
    break
    

CS322
강의 0.09966799462495568
널널 -0.761594155955765
성적 -0.8336546070121552


In [140]:
def append_noun(s):
    return "('%s','Noun')" % (s)

naver_wv_categories = ["강의", "부담", "성적"]
wv_update = {}
for i in range(CATEGORY_COUNT):
    for k,v in scores[i].items():
        if append_noun(k) in pos_vectors.vocab:
#             print(k,naver_wv_categories[i])
#             print(pos_vectors.similarity(append_noun(k),append_noun(naver_wv_categories[i])))
            wv_update[k] = pos_vectors.similarity(append_noun(k),append_noun(naver_wv_categories[i]))
        

c:\users\minkyu\appdata\local\programs\python\python35\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [141]:
review_update = {}
for i in range(len(f)):
    line = f[i]
    course = json.loads(line)

    for i in range(CATEGORY_COUNT):
        if course[subjects_str[i]] == "?":
            continue
        for k,v in scores[i].items():
            if k in course["review_text"].split():
                if k not in review_update:
                    review_update[k] = 0
                review_update[k] += calc_weight(course[subjects_str[i]])*100.0/len(course["review_text"])
        

In [142]:
aggregated_scores = [{} for i in range(CATEGORY_COUNT)]
for i in range(CATEGORY_COUNT):
    for k,v in scores[i].items():
        aggregated_scores[i][k] = v
        if k in review_update:
            aggregated_scores[i][k] += review_update[k]*0.1
        
        if k in wv_update:
            aggregated_scores[i][k] += wv_update[k]
            
            

In [143]:
def calc_aggregated_score(s, subject_str):
    result = 0
    for k,v in aggregated_scores[subject_mp[subject_str]].items():
        if k in s:
            result += v
#     result=1.0/(1+math.exp(-result))
    result=math.tanh(result)
    return result

def calc_aggregated_score_sum(column_name,i):
    course = json.loads(f[i])
    return calc_aggregated_score(course["review_text"], column_name)

# print(subjects_str)

In [144]:
for i in range(len(f[:1])):
    course = json.loads(f[i])
    print(course)

{'추천': '0', 'url_id': 2888, 'review_text': '오랜만에 한국어 강의가 개설되어 재미있게 들었습니다.학점은 한국어 강의니까 빡시게 주신다고 했는데 예상했던 만큼 딱 나왔습니다. 다른 전공과 크게 다르지 않은 수준.거의 매주 손으로 풀어야하는 숙제가 나오고 프로그래밍 프로젝트도 4~5개 있었는데 미루지 않고 꼬박꼬박 하면 할만합니다. 코딩 플젝은 스펙이 매우 자유로워서 당황스러울 수 있는데 그만큼 채점도 너그럽게 하시는 것 같습니다. 시험 공부할 양도 아주 많진 않아요.교수님께서 강의를 잘하셔서(특히 한국어강의여서) 집중해서 듣기만 하면 이해도 잘되고 남는것도 많았습니다. 수업중 딴얘기도 종종 하시는데 저에게는 개인적으로 유익했지만 이건 호불호가 갈리는듯...', 'course_name': '형식언어및오토마타', '성적': '?', 'semester': '가을', 'course_code': 'CS322', 'year': '2012', '강의': '?', '널널': '?', '종합': '?', 'prof_name': '최광무'}


In [165]:
pos = {}
neg = {}
PNS = {}
course_list = []
print(subjects_str)

for i in range(len(f[:])):
    course = json.loads(f[i])
#     print(course["course_code"]+"\t",end="")
    if course["course_code"] not in pos:
        pos[course["course_code"]] = [[0,0],[0,0],[0,0]]
        neg[course["course_code"]] = [[0,0],[0,0],[0,0]]
        PNS[course["course_code"]] = [[0,0],[0,0],[0,0]]#[0,0,0]
        course_list.append(course["course_code"])
    for x in subjects_str:
#             out.write(str(calc_aggregated_score_sum(x,i))+"\t")
#         print(str(calc_aggregated_score_sum(x,i))+"\t",end="")
        sc = calc_aggregated_score_sum(x,i)
        if sc>0:
            pos[course["course_code"]][subject_mp[x]][0] += sc*10
            pos[course["course_code"]][subject_mp[x]][1] += 1
        else:
            neg[course["course_code"]][subject_mp[x]][0] -= sc*10
            neg[course["course_code"]][subject_mp[x]][1] += 1
    print("")
    for x in subjects_str:
#         pos_vir = pos[course["course_code"]][subject_mp[x]][1]
#         neg_vir = neg[course["course_code"]][subject_mp[x]][1]
#         if pos_vir == 0 and neg_vir == 0:
#             pos_vir = 1
        pos_sum = pos[course["course_code"]][subject_mp[x]][0]
#         pos_cnt = pos[course["course_code"]][subject_mp[x]][1]
        neg_sum = neg[course["course_code"]][subject_mp[x]][0]
#         neg_cnt = neg[course["course_code"]][subject_mp[x]][1]
        PNS[course["course_code"]][subject_mp[x]][0] = pos_sum-neg_sum

['강의', '널널', '성적']






























































































































































































































































































































































































































































































































































































































































































































In [166]:
print(neg["CS322"])

[[24.571523594016867, 12], [88.46827293831832, 22], [76.59449917120381, 20]]


In [167]:
with open("mingyu.txt","w",encoding="utf-8") as out:
    for k,v in pos.items():
        print(k)
        print(*pos[k])
        print(*neg[k])
        print(*PNS[k])
    
    

CS493
[7.187341914478964, 1] [0, 0] [0, 0]
[0, 0] [0.0, 1] [0.0, 1]
[7.187341914478964, 0] [0.0, 0] [0.0, 0]
CS510
[14.902220111725423, 2] [0, 0] [0, 0]
[0.0, 1] [12.077448792468644, 3] [0.0, 3]
[14.902220111725423, 0] [-12.077448792468644, 0] [0.0, 0]
CS454
[28.55591915234894, 3] [0, 0] [0, 0]
[0, 0] [18.12547935323792, 3] [10.346262247718112, 3]
[28.55591915234894, 0] [-18.12547935323792, 0] [-10.346262247718112, 0]
CS206
[189.09449613667977, 32] [83.65962060847681, 16] [86.40497365223166, 19]
[72.3444301147268, 34] [256.4028036165729, 50] [136.80577306731826, 47]
[116.75006602195296, 0] [-172.74318300809608, 0] [-50.400799415086595, 0]
CS492
[71.97698077539229, 11] [17.4577992254933, 4] [33.09362519827306, 7]
[20.934165032791746, 17] [82.03911319618634, 24] [37.202221490511334, 21]
[51.042815742600546, 0] [-64.58131397069303, 0] [-4.108596292238275, 0]
CS548
[5.370495669980353, 1] [13.14508967975661, 2] [3.812048023108864, 1]
[0.0, 1] [0, 0] [0.0, 1]
[5.370495669980353, 0] [13.14508

In [169]:
for k,v in pos.items():
    with open("../data/"+k+".csv","w",newline='') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerow(["Course","Pos","Neg","PNS"])
        for subject in subjects_str :
            if subject == "강의":
                line = ["Lecture",str(pos[k][0][0]),str(neg[k][0][0]),str(PNS[k][0][0])]
            elif subject == "널널":
                line = ["Load",str(pos[k][1][0]),str(neg[k][1][0]),str(PNS[k][1][0])]
            else:
                line = ["Grade",str(pos[k][2][0]),str(neg[k][2][0]),str(PNS[k][2][0])]
            writer.writerow(line)

In [170]:
for i in range(len(f[:])):
    course = json.loads(f[i])
    str1 = course["course_code"] + course["year"] + course["semester"]
    print(str1)
    break
    

CS3222012가을


In [174]:
pos1 = {}
neg1 = {}
PNS1 = {}

for i in range(len(f[:])):
    course = json.loads(f[i])
    key1 = course["course_code"] +" "+ course["year"] +" "+ course["semester"]
    if course["semester"]!="봄" and course["semester"]!="가을":
        pass
    else:
        print(course["course_code"]+"\t",end="")
        if key1 not in pos1:
            pos1[key1] = [[0,0],[0,0],[0,0]]
            neg1[key1] = [[0,0],[0,0],[0,0]]
            PNS1[key1] = [[0,0],[0,0],[0,0]]#[0,0,0]
        for x in subjects_str:
    #             out.write(str(calc_aggregated_score_sum(x,i))+"\t")
            print(str(calc_aggregated_score_sum(x,i))+"\t",end="")
            sc = calc_aggregated_score_sum(x,i)
            if sc>0:
                pos1[key1][subject_mp[x]][0] += sc*10
                pos1[key1][subject_mp[x]][1] += 1
            else:
                neg1[key1][subject_mp[x]][0] -= sc*10
                neg1[key1][subject_mp[x]][1] += 1
        print("")
        for x in subjects_str:
            pos_vir1 = pos1[key1][subject_mp[x]][1]
            neg_vir1 = neg1[key1][subject_mp[x]][1]
            if pos_vir1 == 0:
                pos_vir1 = 1
            if neg_vir1 == 0:
                neg_vir1 = 1
            pos_sum1 = pos1[key1][subject_mp[x]][0]
            pos_cnt1 = pos1[key1][subject_mp[x]][1]
            neg_sum1 = neg1[key1][subject_mp[x]][0]
            neg_cnt1 = neg1[key1][subject_mp[x]][1]
            PNS1[key1][subject_mp[x]][0] = pos_sum1-neg_sum1
            
            
            

CS322	0.44171825991105995	-0.993671458644531	-0.7605653232742908	
CS493	0.7187341914478964	0.0	0.0	
CS322	0.23188164037201675	0.0	0.5370495669980353	
CS311	0.965594993794088	-0.9990999415105588	-0.9856864444027524	
CS230	0.8336546070121552	-0.8920476803843821	-0.5370495669980353	
CS300	0.0	0.10112167595257296	-0.19596357984718382	
CS109	0.05408514398776927	0.0	0.09966799462495582	
CS206	-0.37668098171047204	0.5370495669980353	0.0	
CS204	0.5259612776291301	-0.9183003715935452	0.0	
CS220	0.9769069984876954	0.8828850799758959	0.9592466461956096	
CS330	-0.37284367395320756	-0.9811481134972714	-0.3386865799281181	
CS448	-0.5068555079353851	0.0	0.0	
CS211	0.5842192731929995	-0.004279300100870426	0.0	
CS230	0.9531724441745069	-0.9781843667293797	0.0	
CS202	0.9705418327323984	-0.6992147364151174	0.0	
CS202	0.990790821492823	-0.8945203211004721	-0.6537777386316032	
CS109	0.137559399900972	-0.10488396024649965	0.0	
CS330	-0.37284367395320756	-0.6038724396234322	0.0	
CS311	0.5259612776291301	-0.6

CS300	0.0	0.0	0.0	
CS206	0.6281809015165705	-0.8474137860198121	0.547122241867763	
CS101	0.1533130081576913	0.0	0.0	
CS230	0.0	0.0	0.0	
CS311	-0.19737532022490406	0.3464362947240175	-0.5927326264238819	
CS322	0.0	-0.5886392305569499	0.0	
CS330	-0.7525667271990237	-0.9997789992180007	-0.5756755532415789	
CS380	0.5842192731929995	0.0	0.09955666369698038	
CS311	0.8799234438070063	-0.3877226772995129	0.18859794021499138	
CS300	-0.3839066308904931	0.5770417487359215	-0.6537777386316032	
CS204	0.0	-0.8765542143061794	0.06533226716155516	
CS101	0.0	-0.8941187021626574	0.0	
CS202	0.9095126008315194	-0.9904233789914837	-0.6537777386316032	
CS380	0.5842192731929995	0.0	0.0	
CS408	0.0	-0.4865445607922761	0.0	
CS220	-0.5890075571862953	-0.7953256421607612	-0.5370495669980353	
CS230	0.09219412530814193	0.8930802483103064	0.9356505237079596	
CS442	0.7181209465014091	-0.9246939797926632	0.7631663230287735	
CS350	0.982945980425641	0.0	0.0	
CS311	0.07960119957788686	0.0	0.0	
CS230	-0.7973913010715209	-

CS492	0.5370495669980353	-0.6065492732609231	0.0	
CS311	0.5370495669980353	-0.9902422807400852	-0.46220558611171897	
CS420	0.0	0.4736378732464207	0.0	
CS520	-0.028074229067659887	-0.6429263734773529	0.0	
CS548	0.0	0.8512376630961339	0.0	
CS101	0.5259612776291301	0.0	0.19636437245805743	
CS311	0.0	0.641447847790136	0.0	
CS454	0.9531724441745069	-0.19662563781523365	0.0	
CS320	0.1759200379591185	-0.2291307691859777	0.0	
CS470	0.7181209465014091	0.6661303487209524	0.7631663230287735	
CS320	0.0	0.0	0.0	
CS320	-0.6640367702678489	0.0	0.0	
CS206	0.0	-0.30990026800430853	-0.9334286801019395	
CS470	-0.9921089863124936	-0.9886009050964689	-0.3825215195498546	
CS442	0.5370495669980353	-0.8619797365264749	-0.5370495669980353	
CS341	0.1929182133357421	-0.7724573990518617	0.5370495669980353	
CS476	-0.29131261245159096	-0.9910427207132121	-0.5370495669980353	
CS374	-0.4575675276304022	-0.8472165026745874	-0.6537777386316032	
CS592	0.0	0.0	0.0	
CS330	-0.9782841127612344	-0.9999708938218049	-0.5756755

In [175]:
with open("mingyu.txt","w",encoding="utf-8") as out:
    for k,v in pos1.items():
        print(k)
        print(v)

CS311 2013 봄
[[6.3432162776831555, 1], [0, 0], [7.499840953501835, 1]]
CS448 2017 봄
[[0, 0], [0, 0], [0.9966799462495581, 1]]
CS492 2018 봄
[[26.399273033182098, 4], [9.957958271991465, 3], [13.065032125628937, 3]]
CS380 2016 봄
[[20.863107372917593, 3], [0.8484421012051171, 1], [0.9955666369698037, 1]]
CS320 2018 봄
[[10.224297188386512, 2], [4.534757791414129, 2], [0, 0]]
CS322 2014 가을
[[2.4305306296403266, 1], [0, 0], [0, 0]]
CS101 2014 가을
[[8.004990217606297, 1], [0, 0], [0, 0]]
CS206 2013 봄
[[0, 0], [0, 0], [0, 0]]
CS320 2014 봄
[[6.765227072661838, 1], [1.6723998771809756, 1], [3.812048023108864, 1]]
CS350 2018 봄
[[13.680051020655203, 3], [0, 0], [0, 0]]
CS350 2015 봄
[[0, 0], [0, 0], [0, 0]]
CS672 2012 가을
[[9.665229730582555, 1], [0, 0], [0, 0]]
CS320 2015 봄
[[0, 0], [0, 0], [0, 0]]
CS206 2014 봄
[[0, 0], [0, 0], [0, 0]]
CS360 2013 가을
[[9.852144111072306, 1], [0, 0], [7.631663230287735, 1]]
CS320 2013 봄
[[10.220615747688012, 2], [0, 0], [0, 0]]
CS109 2016 봄
[[34.86133134280648, 4], [9

In [176]:
#season : 11:13 course:0:5, Year:6:10
# course_dict ={} #key:name, value: dict->key:year+semester val:PNS
# for k,v in PNS.items():
#     name = k[0:5]
#     if name not in course_dict:
#         course_dict[name][k[6:]]=v
# print(course_dict)
# for k1,v1 in course_dict.items():
#     #with codecs.open("../data/line/"+k[0:5]+" line.csv","wb","utf-8-sig") as file:
#     #writer = csv.writer(file, delimiter=',')
#     L = [["Categories","Lecture","Load","Grade"]]
#     for k2,v2 in v1.items():
#         L.append([k2,v2[0][0],v2[1][0],v2[2][0]])
# print(L)

In [180]:
for course_tmp in course_list:
    L =[["Categories","Lecture","Load","Grade"]]
    for i in range(2000,2019):
        for k1,v1 in PNS1.items():
            if k1[0:5] ==course_tmp and k1[6:10] == str(i):
                if k1[11:] == "봄":
                    L.append([k1[6:10]+" Spring",v1[0][0],v1[1][0],v1[2][0]])
                if k1[11:] == "가을":
                    L.append([k1[6:10]+" Fall",v1[0][0],v1[1][0],v1[2][0]])    
    if(len(L)>2):
        for idx in range(1,len(L)-1):
            if int(L[idx][0][0:4])==int(L[idx+1][0][0:4]) and L[idx][0][5:]=="Fall" and L[idx+1][0][5:]=="Spring":
                tmp = L[idx]
                L[idx] = L[idx+1]
                L[idx+1] = tmp
    with open("../data/line/"+course_tmp+" line.csv","w",newline='') as file:
        writer = csv.writer(file, delimiter=',')
        for i in range(len(L)):
            writer.writerow(L[i])
        
        